# Quantifying Cryptocurrency Price Movements 
#### Market vs. Currency

Part of deciding which cryptocurrencies are to invest in or predicting the prices of cryptocurrencies is determining how much value to place on the technical analysis of the cryptocurrency itself an how much value to place on analysis of the market.

When cryptocurrencies are moving as a result of a bullish or bearish market, they tend to be moving in the same direction at a similar pace. This means that if most of the cryptocurrencies are moving similarly, it is more likely that this movement is related to the market more than it is the investment potential (or FOMO, FUD or whatever other reason people have for buying or selling) of the individual cryptocurrency. 
So in this notebook, we look at quantifying the similarities/differences in the movements of cryptocurrencies. 

Once we have an idea of how various cryptocurrencies are tending to move, we can make a more informed decision on what to invest in, based on how much risk we are comfortable with and how much returns we can expect. As modern portfolio theory states, higher risk is associated with greater probability of higher return and lower risk with a greater probability of smaller return.

In the first sections we will examine daily price changes and variance. In further sections we will build a basic market index for cryptocurrencies, and also a volatility index (not yet implemented) and see how it compares to regular stock market volatility. 

Lastly (not yet implemented), I would like to examine how to put together a cryptocurrency portfolio based on optimising for return and minimising risk (as far as possible with cryptos!).

#### Contents
   * [Correlation Heatmap](#correlation_heatmap)
   * [Daily Percent Change](#daily_percent_change)
   * [Calculating Variance](#calculating_variance)
   * [Crypto20 Market Cap Index](#market_cap_index)
   * [Crypto20 Equal Weight Index](#equal_weight_index)
   
This is a work in progress so it may still be a little rough around the edges. Please feel free to comment and leave any feedback.

In [200]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

sns.set(rc={"figure.figsize": (20,10), "axes.titlesize" : 18, "axes.labelsize" : 12, 
            "xtick.labelsize" : 14, "ytick.labelsize" : 14 }, 
        palette=sns.color_palette("Blues_r", 20))

import warnings
warnings.filterwarnings('ignore')

In [201]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
all_currencies_df = pd.read_csv('../input/all_currencies.csv', parse_dates=['Date'], date_parser=dateparse)

In [202]:
all_currencies_df.sample(5)

Lets get the rank for each crypto by market cap, so that we can include the top x in our analyses. The cryptocurrency space is so volatile that if we consider many more than the top 50 or so, there will simply be too many changes on a daily basis, with new ones coming in, scams coins disappearing, etc.

In [203]:
# Get rank by marketcap
last_date = max(all_currencies_df.Date)
last_date_currencies_df = all_currencies_df[all_currencies_df['Date'] == last_date]

last_date_currencies_df['rank'] = last_date_currencies_df['Market Cap'].rank(method='dense', ascending=False)

all_currencies_df = pd.merge(left=all_currencies_df,right=last_date_currencies_df[['Symbol', 'rank']], how='left', left_on='Symbol', right_on='Symbol')


In [204]:
# Initially we will just work with the top 50 cryptos with data starting from 2017 to speed things up
top50_currencies_df = all_currencies_df[all_currencies_df['rank'] <= 50]
#top50_currencies_df = top50_currencies_df[(top50_currencies_df['Date'] >= '2017-01-01')]

## Correlation Heatmap<a class="anchor" id="correlation_heatmap"></a>

Plotting a quick correlation heatmap should be useful for getting a quick overview of just how much cryptocurrency prices are moving "in sync" with eachother.

In [205]:
top50_pivot_df = top50_currencies_df.pivot(index='Date', columns='Symbol', values='Close')
top50_corr = top50_pivot_df.corr()

cmap = sns.diverging_palette(240, 10,sep=20, as_cmap=True)

plt.figure(figsize = (16,16))
plt.tight_layout()
sns.heatmap(top50_corr, 
            xticklabels=top50_corr.columns.values,
            yticklabels=top50_corr.columns.values, 
            cmap=cmap, vmin=-1, vmax=1, annot=False, square=True)

Overall it looks like a very highly correlated space. A few notable exceptions are BCD (Bitcoin Diamond), BTG (Bitcoin Gold), CNX (Cryptonex), HSR (Hshare), POA (POA Network). I have not heard of any of these except for Bitcoin Gold. It could be that they have only recently come into existence and therefore are experiecing the initial insane volatility followed by being newly listed. Lets have a look at their stats.

In [208]:
latest_date = max(top50_currencies_df['Date'])
top50_currencies_latest_df = top50_currencies_df[top50_currencies_df['Date'] == latest_date]
corr_outliers_df = top50_currencies_latest_df[top50_currencies_latest_df['Symbol'].isin(['BCD', 'BTG', 'CNX', 'HSR', 'POA'])]
corr_outliers_df

In [209]:
top50_start_dates = top50_currencies_df.loc[top50_currencies_df.groupby('Symbol')['Date'].idxmin()]
corr_outliers_df = top50_start_dates[top50_start_dates['Symbol'].isin(['BCD', 'BTG', 'CNX', 'HSR', 'POA'])]
corr_outliers_df

It looks like our theory is plausible - all five currencies that look like they are breaking the trends have their start date later than mid 2017 and could still be in their "new currency going insane" phase.

Lets narrow our analysis down to the top 20 in the hope of eliminating extremely new currencies and do the plot again. We will also set the minimum value to 0 in order to get a larger colour differentiation between correlations.

In [214]:
top20_currencies_df = all_currencies_df[all_currencies_df['rank'] <= 20]

top20_pivot_df = top20_currencies_df.pivot(index='Date', columns='Symbol', values='Close')
top20_corr = top20_pivot_df.corr()

cmap = sns.diverging_palette(240, 10,sep=20, as_cmap=True)

plt.figure(figsize = (16,16))
plt.tight_layout()
sns.heatmap(top20_corr, 
            xticklabels=top20_corr.columns.values,
            yticklabels=top20_corr.columns.values, 
            cmap=cmap, vmin=0, vmax=1, annot=True, square=True)

Using this zoomed-in colour differentiation we now notice the next largest "trend-breaker", ZEC. Which is not really a trend-breaker, still having a correlations between 0.58 and 0.91 with other top 20 currencies.

### Daily Percent Change <a class="anchor" id="daily_percent_change"></a>

Ok, this is a little disappointing as I was hoping to be able to identify more independent movement between cryptocurrencies in order to fulfil the goals of this notebook. However, lets proceed anyway and try to get a better picture of the volatility within the crypto space.

In [196]:
# Reshape the dataframe to multiindex by Date and currency
top50_currencies_df.set_index(['Symbol', 'Date'], inplace=True) # Or should it be the other way around?
top50_currencies_df.head()

In [131]:
# Get daily percent change in close price
top50_currencies_df['pct_change'] = top50_currencies_df['Close'].groupby(level=0).pct_change()
top50_currencies_df.head()

In [132]:
# Plot daily percentage change for BTC
btc_df = top50_currencies_df.xs('BTC')
btc_df.head()

In [215]:
btc_df['pct_change'].plot(figsize=(16,8), title='BTC Daily Percentage Change')
plt.savefig('bitcoin_daily_pct_change.png')

### Calculating Variance <a class="anchor" id="calculating_variance"></a>

Lets go with a 30 day variance calculation. That means that assigned to each day will be the average variance of the movements of the 50 chosen stocks over the last 30 days. This should give us a longer-term view of the market vs. fundamentals tension.

Method:
 - Calculate the mean price for the number of periods/observations
 - Determine each period's deviation (close minus average price)
 - Variance = sum of the squared deviations, divided by the number of observations
 (- Standard deviation = square root of variance)

In [134]:
# calculate mean price over the past 30 days
top50_currencies_df['ma_30'] = top50_currencies_df['pct_change'].rolling(window=30).mean()

top50_currencies_df['deviation'] = top50_currencies_df['pct_change'] - top50_currencies_df['ma_30']

top50_currencies_df['variance'] = top50_currencies_df['deviation']**2 / 30

top50_currencies_df.sample(5)

In [135]:
top50_average_variance_df = top50_currencies_df['variance'].groupby('Date').mean()
top50_average_variance_df.plot(figsize=(16,8), title='Daily Average Variance of Top50 Currencies')

In [136]:
# Lets look at just the past year
top50_average_variance_df = top50_average_variance_df.loc['2017-04-01':]
top50_average_variance_df.plot(figsize=(16,8), title='Daily Average Variance of Top50 Currencies from 2017-04-01')

This shows that the variance in stock movements for the last month has been the lowest since mid October last year, indicating that stocks are moving in the same direction more so than in the previous six months. This kind of analysis helps us put weights on the utility of stock-specific data versus market-specific data.

## Crypto20 Market Cap Index <a class="anchor" id="market_cap_index"></a>

Another way to get an easy comparison of how any particular currency is moving relative to the market is to build an index to compare it to. An index will also be able to show trends and changes in investing patterns much better than looking at Bitcoin or a handfull of other currencies on their own, which up until recently might have been fine given Bitcoin's dominance.
 
We can calculate a crypto index, lets make it crypto20 (Because the crypto space is so volatile that taking the top 50 would result in too many changes to cryptocurrencies needing to be included in the index), using weighted average market capitalisation (following the S&P 500 method).
- Get top 20 currencies by marketcap
- For each row (day, currency) calculate the market weight -> #outstanding units * price / (total market cap)
- Calculate an index divisor -> eg: divisor = total market cap / 100 would give us a 100 point starting index 

In [137]:
top20_currencies_df = all_currencies_df[all_currencies_df['rank'] <= 20]
top20_start_dates = top20_currencies_df.loc[top20_currencies_df.groupby('Symbol')['Date'].idxmin()]
start_date = max(top20_start_dates.Date)

In [138]:
# Lets start this index at the earliest date when the top 20 cryptos all existed
top20_start_dates = top20_currencies_df.loc[top20_currencies_df.groupby('Symbol')['Date'].idxmin()]

In [139]:
# Determine index start date
start_date = max(top20_start_dates['Date'])
start_date

In [140]:
# Remove all older data
index_currencies_df = top20_currencies_df[(top20_currencies_df['Date'] >= start_date)]
index_currencies_df.sample()

In [141]:
# Detemine index divisor at the start date 
crypto20_index_divisor = index_currencies_df[index_currencies_df['Date']==start_date]['Market Cap'].sum() / 100
crypto20_index_divisor

In [142]:
# Build the index
def calculate_index_value(df_sub):
    return df_sub['Market Cap'].sum() / crypto20_index_divisor

crypto20_index_df = index_currencies_df.groupby('Date').apply(calculate_index_value)
crypto20_index_df.head()

In [143]:
crypto20_index_df.plot(figsize=(16,8), title='Crypto20 Index')

In [144]:
# Lets see just how much influence BTC has on this index
top20_latest_df = top20_currencies_df[all_currencies_df['Date'] == last_date]
top20_latest_df.sort_values(by='Market Cap',ascending=False).plot(x='Symbol', y='Market Cap', kind='bar', figsize=(16,8))

It looks like BTC dominates the index so much that the rest almost fade into insignificance.

## Crypto20 Equal Weight Index <a class="anchor" id="equal_weight_index"></a>


Although market cap indices are the most popular, it is not always the best way to examine movements, especially when a single currency dominates the whole index. Another option is an equal weight index, where all currencies are weighted equally. Lets build this with the top 20 cryptocurrencies and see what it looks like.

In [145]:
# Detemine initial amount of each currency assuming an index starting value of 100
n = 20
index_currencies_start_df = index_currencies_df[index_currencies_df['Date']==start_date]
index_currencies_start_df['eq_index_holdings'] = 100 / n / index_currencies_start_df['Close']

In [146]:
# Add eq_index_holdings column to index_currencies_df
index_currencies_df = pd.merge(left=index_currencies_df,right=index_currencies_start_df[['Symbol', 'eq_index_holdings']], how='left', left_on='Symbol', right_on='Symbol')
# Calculate the value for each date/Currency
index_currencies_df['eq_index_holdings_value'] = index_currencies_df['eq_index_holdings']*index_currencies_df['Close']

In [147]:
# Build index
crypto20eq_index_df = index_currencies_df.groupby('Date')['eq_index_holdings_value'].sum()
crypto20eq_index_df.head()

Lets plot this index along with the previous one to see how they compare.

In [148]:
with sns.color_palette("Blues", 2):
    ax = crypto20_index_df.plot(figsize=(16,8), title='Crypto20 Index', label='Crypto20 Market Cap Index', legend=True)
    crypto20eq_index_df.plot(ax=ax, label='Crypto20 Equal Weight Index', legend=True)

The market cap index dominated by Bitcoin looks so stable in comparison to the equal weight index, where each top 20 currency has an equal weighting!

## Beta Calculation
A common measure of a stock’s volatility relative to the broader market is known as the stock’s beta. This number compares the movements of an individual security against those of a benchmark index, which is assigned a beta of 1. For example, a stock with a beta value of 1.2 has historically moved 120 percent for every 100 percent move in a benchmark index such as the S&P 500. 
We can apply this to cryptocurrencies and calcualte the beta for a currency by measuring its movement against the Crypto20 Index. To calculate the beta we need to calculate the covariance between the return of the crypto and the market return, and then divide it by the variance of market returns.

Lets use a monthly period for variance calculations and use the same calculation method (previously outlined) for the equal weighted index.


In [149]:
crypto20eq_index_df = crypto20eq_index_df.reset_index()
crypto20eq_index_df.head()

In [150]:

# Crypto - We already have top50_currencies_df['pct_change'] from a previous step

# calculate the market variance based on the Crypto20 Equal Weight Index
crypto20eq_index_df['pct_change'] = crypto20eq_index_df['eq_index_holdings_value'].pct_change()

crypto20eq_index_df['ma_30'] = crypto20eq_index_df['pct_change'].rolling(window=30).mean()

crypto20eq_index_df['deviation'] = crypto20eq_index_df['pct_change'] - crypto20eq_index_df['ma_30']

crypto20eq_index_df['variance'] = crypto20eq_index_df['deviation']**2 / 30
crypto20eq_index_df.tail()

In [151]:
# calculate the covariance between the return of the crypto and the market return

#better way to join than resetting index?
top50_currencies_df = top50_currencies_df.reset_index()

# covariance
# This is probably not the most efficient way of achieving this calculation. If any python experts out there know a better way please let me know. In principle rolling windows across multiple dataframes?
top50_currencies_df =  pd.merge(left=top50_currencies_df,right=crypto20eq_index_df[['Date', 'pct_change', 'ma_30', 'deviation']], how='left', left_on='Date', right_on='Date')
top50_currencies_df['product'] = top50_currencies_df['deviation_x']*top50_currencies_df['deviation_y']


top50_currencies_df.tail()

In [153]:
top50_currencies_df['covariance'] = top50_currencies_df['product'].rolling(window=30).sum()/29
top50_currencies_df.tail()

In [179]:
# Plot the covariance
# Split into top10, middle20, bottom20 because the plots become too cluttered to read otherwise.
top_currencies_df = top50_currencies_df[top50_currencies_df['rank'] <= 10]
mid_currencies_df = top50_currencies_df[(top50_currencies_df['rank'] > 10) & (top50_currencies_df['rank'] <= 30)]
bottom_currencies_df = top50_currencies_df[top50_currencies_df['rank'] > 30]

top_currencies_cov_df = top_currencies_df.pivot(index='Date', columns='Symbol', values='covariance')

with sns.color_palette("husl", 10):
    top_currencies_cov_df.plot(figsize=(16,8), title='Covariance')

In [181]:
mid_currencies_cov_df = mid_currencies_df.pivot(index='Date', columns='Symbol', values='covariance')

with sns.color_palette("husl", 20):
    mid_currencies_cov_df.plot(figsize=(16,10), title='Covariance')

In [183]:
bottom_currencies_cov_df = bottom_currencies_df.pivot(index='Date', columns='Symbol', values='covariance')

with sns.color_palette("husl", 20):
    bottom_currencies_cov_df.plot(figsize=(16,10), title='Covariance')

In the stock market, a strong emphasis is placed on reducing the risk amount taken on for the same amount of return. When constructing a portfolio, an analyst will select stocks that will work well together. This usually means that these stocks do not move in the same direction. This could prove quite difficult for the cryptocurrency market, as it currently seems like most of the top cryptocurrencies move with the market. They have yet to develop their own strong identities.

This is a work in progress. Please leave a comment and let me know if you think of any other ways this can be improved.